# Classifying Disaster Tweets

Kaggle Dataset Download API Command:

kaggle competitions download -c nlp-getting-started

I will classify a tweet as either a 'Disaster Tweet' or 'Non-Disaster Tweet'.

##Project Outline:

1) Download the dataset

2) Explore/Analyze the data

3) Preprocess and organize the data

4) Classify using Vader

5) Classify using Bag of Words

6) Classify using Hugging Face

## Download the Dataset

1) Install required libraries

2) Import required libraries

3) Upload Data from Google Drive


#### Install Required Libraries

In [7]:
!pip install scikit-learn numpy transformers datasets > /dev/null 2>&1

#### Import Required Libraries

In [13]:
# handeling data
import numpy as np
import pandas as pd

# graphing data
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go

# downloading data
from google.colab import drive

# splitting data
from sklearn.model_selection import train_test_split

# training the data
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# suppress warnings
import warnings
import os
import logging
from transformers import logging as hf_logging

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0 = all logs, 3 = only errors
tf.get_logger().setLevel(logging.ERROR)

# Suppress HuggingFace Transformers warnings
hf_logging.set_verbosity_error()

# Suppress the specific Hugging Face warning
warnings.filterwarnings("ignore")

####Upload Data from Google Drive

In [9]:
# Mount google drive to store Kaggle API for future use
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# read in the data
HF_train = pd.read_csv('/content/drive/My Drive/ucla.edu_folder/Disaster_Tweets/train_df.csv').copy()
HF_test = pd.read_csv('/content/drive/My Drive/ucla.edu_folder/Disaster_Tweets/test_df.csv').copy()

##**Find Sentiment with Hugging Face**

https://blog.tensorflow.org/2019/11/hugging-face-state-of-art-natural.html

### Split / Preprocess Training and Testing Data

In [11]:
# create a method that will split the data into training and testing sets
def split_data_frame(data_frame, target, test_size, shuffle=True):
  X = data_frame.drop(columns=target)
  y = data_frame[target]
  X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=test_size, shuffle=shuffle)
  print(f'X_train: {X_train.shape}, X_valid: {X_valid.shape}, y_train: {y_train.shape}, y_valid: {y_valid.shape}')
  return X_train, X_valid, y_train, y_valid

X_train, X_valid, y_train, y_valid = split_data_frame(data_frame=HF_train, target='target', test_size=0.10)

X_train: (6851, 1), X_valid: (762, 1), y_train: (6851,), y_valid: (762,)


In [14]:
# tokenizer from a pretrained bert model
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

# create a tokenizer method
def tokenize_reviews(X_train, X_valid, y_train, y_valid, X_test):
  # Tokenize the reviews
  token_train_data = tokenizer(X_train['text'].to_list(), return_tensors='np', padding=True)
  token_valid_data = tokenizer(X_valid['text'].to_list(), return_tensors='np', padding=True)
  token_test_data = tokenizer(X_test['text'].to_list(), return_tensors='np', padding=True)

  # convert labels to a numpy array
  train_labels = np.array(y_train)
  valid_labels = np.array(y_valid)

  return token_train_data, token_valid_data, train_labels, valid_labels, token_test_data

In [ ]:
token_train_data, token_valid_data, train_labels, valid_labels, token_test_data = tokenize_reviews(X_train, X_valid, y_train, y_valid, HF_test)

In [ ]:
token_train_data['input_ids'][0]

array([  101,  9294,  3048, 24819,   143,  2591,  2428, 19556, 15609,
       14962,  2240,   141,  9949, 15681,   156, 19556, 23314, 17145,
        7462, 10997,  9244, 12507, 24772, 12649,   139, 22027, 16830,
        2346,   143, 27073, 15928,   145,  9565, 20595, 13360,  2036,
        1139,  2869,  2298,  1139,  4329,  1766,  2027,   102,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0])

In [ ]:
token_test_data['input_ids'][0]

array([ 101, 2066, 2171,  170, 6434, 1610, 5683,  102,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

## Create and Train baseline model
1) Create the model

2) Compile the model

3) Train the model

4) Save the Model

5) Plot Training Accuracy / Loss

### Create the model

In [ ]:
# create a method to load the pre-trained model
def load_pre_trained_model(model_name, NUM_LABELS):
  # load the pretrained model 'bert-base-cased'
  model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS)

  return model

In [ ]:
model = load_pre_trained_model('bert-base-cased', 2)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

### Compile the model

In [ ]:
# create a method to create the loss and compile the model
def loss_and_compile(model):
  # create the loss
  loss = tf.keras.losses.SparseCategoricalCrossentropy(filter)

  # compile the model
  model.compile(optimizer=tf.keras.optimizers.Adam(5e-6), loss=loss, metrics=['accuracy'])

  return model

In [ ]:
model_1 = loss_and_compile(model)

### Train the model

In [ ]:
# create a method to train the model
def fit_model(model, BATCH_SIZE, EPOCHS, train_data, train_labels, val_data, val_labels):
  # fit the model
  history = model.fit(dict(train_data),
            train_labels,
            validation_data=(dict(val_data), val_labels),
            batch_size=BATCH_SIZE,
            epochs=EPOCHS)

  return model, history

In [ ]:
model, history = fit_model(model_1, 64, 100, token_train_data, train_labels, token_valid_data, valid_labels)

Epoch 1/100
108/108 [==============================] - 175s 1s/step - loss: 0.5483 - accuracy: 0.7365 - val_loss: 0.4349 - val_accuracy: 0.8294
Epoch 2/100
108/108 [==============================] - 107s 992ms/step - loss: 0.4209 - accuracy: 0.8244 - val_loss: 0.3929 - val_accuracy: 0.8320
Epoch 3/100
108/108 [==============================] - 107s 992ms/step - loss: 0.3739 - accuracy: 0.8483 - val_loss: 0.3864 - val_accuracy: 0.8360
Epoch 4/100
108/108 [==============================] - 107s 994ms/step - loss: 0.3349 - accuracy: 0.8720 - val_loss: 0.4144 - val_accuracy: 0.8176
Epoch 5/100
108/108 [==============================] - 107s 995ms/step - loss: 0.3045 - accuracy: 0.8847 - val_loss: 0.4237 - val_accuracy: 0.8255
Epoch 6/100
108/108 [==============================] - 106s 984ms/step - loss: 0.2767 - accuracy: 0.9005 - val_loss: 0.4154 - val_accuracy: 0.8425
Epoch 7/100
108/108 [==============================] - 106s 980ms/step - loss: 0.2471 - accuracy: 0.9139 - val_loss: 0.43

####Save the Model



In [15]:
model.save('/content/drive/My Drive/ucla.edu_folder/Disaster_Tweets/disaster_tweets_model.keras', save_format="keras")

#### Plot Training Accuracy / Loss

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

In [ ]:
def plot_train_loss_and_accuracy(train_loss, train_accuracy):
    # Create a figure with Plotly
    fig = go.Figure()

    # Add traces for training loss and accuracy
    fig.add_trace(go.Scatter(x=list(range(1, len(train_loss) + 1)), y=train_loss, mode='lines', name='Training Loss', yaxis='y1'))
    fig.add_trace(go.Scatter(x=list(range(1, len(train_accuracy) + 1)), y=train_accuracy, mode='lines', name='Training Accuracy', yaxis='y2'))

    # Update layout and labels
    fig.update_layout(title='Training Loss and Accuracy vs. Epochs',
                      xaxis=dict(title='Epochs'),
                      yaxis=dict(title='Training Loss', side='left', showgrid=False),
                      yaxis2=dict(title='Training Accuracy', side='right', overlaying='y', showgrid=False))

    fig.show()

plot_train_loss_and_accuracy(loss, acc)

Notice that the validation loss increases over time, and the validation accuracy decreases over time. Both the accuracy and loss seem to be unstable. There appears to be overfitting in this model. Changing the learning rate and or employing early stopping techniques may improve this model.

In [ ]:
def plot_valid_loss_and_accuracy(val_loss, val_accuracy):
    # Create a figure with Plotly
    fig = go.Figure()

    # Add traces for validation loss and accuracy
    fig.add_trace(go.Scatter(x=list(range(1, len(val_loss) + 1)), y=val_loss, mode='lines', name='Validation Loss', yaxis='y1'))
    fig.add_trace(go.Scatter(x=list(range(1, len(val_accuracy) + 1)), y=val_accuracy, mode='lines', name='Validation Accuracy', yaxis='y2'))

    # Update layout and labels
    fig.update_layout(title='Validation Loss and Accuracy vs. Epochs',
                      xaxis=dict(title='Epochs'),
                      yaxis=dict(title='Validation Loss', side='left', showgrid=False),
                      yaxis2=dict(title='Validation Accuracy', side='right', overlaying='y', showgrid=False))

    fig.show()

plot_valid_loss_and_accuracy(val_loss, val_acc)